In [1]:
import pandas as pd

df = pd.read_csv('winemag-data-130k-v2.csv')

In [2]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
# country points province varity
import numpy as np
coun = df['country'].to_numpy(dtype='str')
provinces = df['province'].to_numpy(dtype='str')
desig = df['designation'].to_numpy(dtype='str')
reg1 = df['region_1'].to_numpy(dtype='str')
reg2 = df['region_2'].to_numpy(dtype='str')
var = df['variety'].to_numpy(dtype='str')
winery = df['winery'].to_numpy(dtype='str')

In [4]:
len(np.unique(var))

708

In [5]:
df.describe()

,Unnamed: 0,points,price
count,129971.000000,129971.000000,120975.000000
mean,64985.000000,88.447138,35.363389
std,37519.540256,3.039730,41.022218
min,0.000000,80.000000,4.000000
25%,32492.500000,86.000000,17.000000
50%,64985.000000,88.000000,25.000000
75%,97477.500000,91.000000,42.000000
max,129970.000000,100.000000,3300.000000


In [6]:
df[df['points']<80]

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery


In [7]:
df['description'].iloc[0]

"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [8]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [9]:
X=enc.fit_transform(df[['country','province','variety','points']])

In [10]:
enc.categories_

[array(['Argentina', 'Armenia', 'Australia', 'Austria',
        'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Canada', 'Chile',
        'China', 'Croatia', 'Cyprus', 'Czech Republic', 'Egypt', 'England',
        'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'India',
        'Israel', 'Italy', 'Lebanon', 'Luxembourg', 'Macedonia', 'Mexico',
        'Moldova', 'Morocco', 'New Zealand', 'Peru', 'Portugal', 'Romania',
        'Serbia', 'Slovakia', 'Slovenia', 'South Africa', 'Spain',
        'Switzerland', 'Turkey', 'US', 'Ukraine', 'Uruguay', nan],
       dtype=object),
 array(['Achaia', 'Aconcagua Costa', 'Aconcagua Valley', 'Aegean',
        'Agioritikos', 'Ahr', 'Alenquer', 'Alentejano', 'Alentejo',
        'Alenteo', 'Algarve', 'Alsace', 'America', 'Amindeo', 'Amyndeon',
        'Andalucia', 'Ankara', 'Apalta', 'Arcadia', 'Arizona', 'Armenia',
        'Atalanti Valley', 'Atlantida', 'Attica', 'Australia Other',
        'Austria', 'Awatere Valley', 'Baden', 'Bairrada', 'Beauj

In [11]:
X

<129971x1199 sparse matrix of type '<class 'numpy.float64'>'
	with 519884 stored elements in Compressed Sparse Row format>

In [12]:
y=np.array(df['price'].fillna(0))

In [13]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [20]:
model = keras.Sequential([
      layers.Dense(100, activation='relu'),
#       layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [21]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9736573832250988959
]


In [22]:
history = model.fit(X,y,verbose=1, epochs=10)

Epoch 1/10
4062/4062 [==============================] - 5s 1ms/step - loss: 15.0481
Epoch 2/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.6780
Epoch 3/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.4794
Epoch 4/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.3752
Epoch 5/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.2957
Epoch 6/10
4062/4062 [==============================] - 4s 1ms/step - loss: 13.2418
Epoch 7/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.1906
Epoch 8/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.1524
Epoch 9/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.1131
Epoch 10/10
4062/4062 [==============================] - 5s 1ms/step - loss: 13.0853


In [25]:
history

AttributeError: 'History' object has no attribute 'score_'